In [1]:
from keras.applications import mobilenet

Using TensorFlow backend.


In [2]:
img_rows = 224
img_cols = 224 
model=mobilenet.MobileNet(weights='imagenet',
                 include_top=False,
                 input_shape = (img_rows, img_cols, 3)
                 )

In [3]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [4]:
#now trainable=False
for layer in model.layers:
    layer.trainable=False
    print(layer.__class__.__name__,layer.trainable)

InputLayer False
ZeroPadding2D False
Conv2D False
BatchNormalization False
ReLU False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
ZeroPadding2D False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
ZeroPadding2D False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
ZeroPadding2D False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU False
DepthwiseConv2D False
BatchNormalization False
ReLU False
Conv2D False
BatchNormalization False
ReLU Fals

In [5]:
def addModel(bottemModel,num_classes,classifier):
    top_model=bottemModel.output
    top_model=Flatten(name='flatten')(top_model)
    top_model=Dense(484,activation='relu')(top_model)
    top_model=Dense(121,activation='relu')(top_model)
    top_model=Dense(num_classes,activation=classifier)(top_model)
    return top_model

In [6]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

In [ ]:
from PIL import Image

In [ ]:
Image.open('./Transfer Learning/Rahul/IMG20180714170243.jpg')

In [7]:
#importing useful libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Flatten
from keras.applications import imagenet_utils

In [8]:
import numpy as np

In [9]:
#training datagenrator
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_data_dir = './dataset_images/train/'
validation_data_dir = './dataset_images/valid/'

train_generotor=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_rows,img_cols),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 )

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        class_mode='categorical',
        shuffle=False)

Found 96 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [10]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

Total params: 3,228,864
Trainable params: 0
Non-trainable params: 3,228,864
_________________________________________________________________


In [11]:
top_model=addModel(model,2,'softmax')

In [12]:
type(top_model)

tensorflow.python.framework.ops.Tensor

In [13]:
from keras.models import Model
new_model=Model(inputs=model.input, outputs=top_model)
print(new_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [14]:
new_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

In [15]:
history=new_model.fit_generator(
    train_generotor,
    epochs=4,
    validation_data=validation_generator,
    validation_steps=6
)
new_model.save('face_usi_mobilet_v2.h5')

Epoch 1/4
3/3 [==============================] - 146s 49s/step - loss: 19.9451 - accuracy: 0.4479 - val_loss: 7.9297 - val_accuracy: 0.5385
Epoch 2/4
3/3 [==============================] - 72s 24s/step - loss: 3.3975 - accuracy: 0.6875 - val_loss: 3.4434 - val_accuracy: 0.4615
Epoch 3/4
3/3 [==============================] - 87s 29s/step - loss: 1.6173 - accuracy: 0.7188 - val_loss: 6.7576 - val_accuracy: 0.5385
Epoch 4/4
3/3 [==============================] - 75s 25s/step - loss: 1.1817 - accuracy: 0.8125 - val_loss: 5.5874 - val_accuracy: 0.5385


In [ ]:
pred=new_model.predict(prepare_image('IMG20190419174241.jpg'))

In [ ]:
pred